# Cross River Bank – Fraud & Risk Analysis

In [ ]:

import pandas as pd
import json
from pathlib import Path

# Load CSV data
customer_df = pd.read_csv("data/customer_table.csv")
loan_df = pd.read_csv("data/loan_table.csv")
transaction_df = pd.read_csv("data/transaction_table.csv")

# Load JSON data
with open("data/behavior_logs.json") as f:
    behavior_logs = pd.json_normalize(json.load(f))
with open("data/customer_feedback.json") as f:
    feedback = pd.json_normalize(json.load(f))

# Convert and clean
customer_df["customer_since"] = pd.to_datetime(customer_df["customer_since"], errors='coerce')
behavior_logs["timestamp"] = pd.to_datetime(behavior_logs["timestamp"], errors='coerce')
behavior_logs["session_duration"] = pd.to_numeric(behavior_logs["session_duration"], errors='coerce')
feedback["sentiment_score"] = pd.to_numeric(feedback["sentiment_score"], errors='coerce')
feedback["escalation_flag"] = feedback["escalation_flag"].astype(str).str.lower() == 'true'

# Structured Analysis
loan_merged = loan_df.merge(customer_df, on="customer_id", how="left")
full_merged = transaction_df.merge(loan_df, on=["loan_id", "customer_id"], how="left").merge(customer_df, on="customer_id", how="left")

high_risk_customers = loan_merged[(loan_merged["default_risk"] == "High") & (loan_merged["credit_score"] < 600)]

loan_purpose_summary = loan_df.groupby("loan_purpose").agg(
    loan_count=pd.NamedAgg(column="loan_id", aggfunc="count"),
    total_loan_amount=pd.NamedAgg(column="loan_amount", aggfunc="sum")
).sort_values(by="total_loan_amount", ascending=False)

high_value_txns = full_merged[full_merged["transaction_amount"] > 0.3 * full_merged["loan_amount"]]

loyal_customers = customer_df[customer_df["customer_since"] <= pd.Timestamp.now() - pd.DateOffset(years=5)]

credit_loan_avg = loan_merged.groupby(pd.cut(loan_merged["credit_score"], bins=[300, 500, 650, 750, 850])).agg(
    avg_loan_amount=pd.NamedAgg(column="loan_amount", aggfunc="mean"),
    count=pd.NamedAgg(column="loan_id", aggfunc="count")
)

age_loan_distribution = loan_merged.groupby(pd.cut(loan_merged["age"], bins=[18, 30, 45, 60, 100])).agg(
    total_loan_amount=pd.NamedAgg(column="loan_amount", aggfunc="sum"),
    loan_count=pd.NamedAgg(column="loan_id", aggfunc="count")
)

# Unstructured Analysis
missed_payments = behavior_logs[behavior_logs["action"] == "Missed Payment"]
missed_counts = missed_payments.groupby("customer_id").size().reset_index(name="missed_payment_count")

sentiment_summary = feedback.groupby("customer_id").agg(
    avg_sentiment_score=pd.NamedAgg(column="sentiment_score", aggfunc="mean"),
    feedback_count=pd.NamedAgg(column="feedback_text", aggfunc="count"),
    escalations=pd.NamedAgg(column="escalation_flag", aggfunc="sum")
).reset_index()

risk_df = missed_counts.merge(sentiment_summary, on="customer_id", how="outer").fillna(0)
risk_df["composite_risk_score"] = (
    risk_df["missed_payment_count"] * 2 +
    (1 - risk_df["avg_sentiment_score"]) * 3 +
    risk_df["escalations"]
)
top_risky_customers = risk_df.sort_values(by="composite_risk_score", ascending=False).head(20)
top_risky_customers
